In [7]:
from itertools import product

***Multiplicity of the sign and trivial representations of $S_n$ in $W_{(k|l)}(\mathbb{C}^n)$ using the combinatorial interpretation***

In [8]:

def sgn_mult_hook(k,l,n):
    """Finds the multiplicity of the sign representation of S_n in W_(k|l)(C^n) using the combinatorial interpretation"""
    sm=0
    Par_sub= [x for x in Partitions(k) if len(x)<n+1]
    for x in Par_sub:
        if len(x.removable_cells())>0 and x.removable_cells()[0][0]==0:
            m=len(x.removable_cells())-1
        else:
            m=len(x.removable_cells())
        m= m+1*(n-len(x)>0)
        sm=sm+binomial(m,n-l-1)
    return sm

def triv_mult_hook(k,l,n):
    """Finds the multiplicity of the trivial representation of S_n in W_(k|l)(C^n) using the combinatorial interpretation"""
    sm=0
    Par_sub= [x for x in Partitions(k) if len(x)<n+1]
    for x in Par_sub:
        y= Partition([i+1 for i in x]+[1]*(n-len(x)))
        m=len(y.removable_cells())
        if max(y)-1 in y:
            sm=sm+binomial(m-1,l)+binomial(m-2,l)
        else:
            sm=sm+binomial(m-1,l)
    return sm

***Code written by Amritanshu Prasad to calculate moments and signed moments***

In [9]:
def next_vector(current, maxvec):
    """
    Return the next vector after ``current`` in lex bounded by ``max``.
    
    INPUT:

    - ``current`` -- a tuple of non-negative integers.

    - ``maxvec`` -- a tuple of integers of the same length as ``current``,
      lexicographically less than ``current``.

    OUTPUT: the next tuple of integers componentwise less than or equal
    to ``current`` in lexicographic order.

    EXAMPLE::

    sage: next_vector((0,0,1), (2,1,1))
    (0,1,0)
    sage: next_vector((1,0,1), (2,0,1))
    (2,0,0)
    """
    if current>=maxvec:
        raise ValueError("``current`` should be less than ``maxvec`` in lexicographic order")
    l = len(current)
    for i in range(l):
        if current[l-i-1]<maxvec[l-i-1]:
            return current[:l-i-1]+(current[l-i-1]+1,)+(0,)*(i)
    # else:
    #     raise ValueError("maxvec should be greater than current in lexicographic order")


def vecpar(vec, minvec=None):
    """
    Iterate over distinct part vector partitions of ``vec``.

    INPUT:

    - ``vec`` -- tuple of non-negative integers.

    - ``minvec`` -- a tuple of nonnegative integers of the same length
      as ``vec`` and componentwise less than or equal to ``vec``.
    """
    l = len(vec)
    if minvec is None:
        minvec = (0,)*l
    parts = [minvec]
    res = [tuple((vec[i]-parts[-1][i] for i in range(l)))]
    current = [minvec]
    while True:
        try:
            v = next_vector(current[-1],res[-1])
            res.append(tuple(res[-1][i]-v[i] for i in range(l)))
            parts.append(v)
            current.append(v)
            if not any(res[-1]):
                yield VectorPartition(parts)
                current.pop()
                res.pop()
                parts.pop()
                if len(current)>1:
                    current.pop()
                    current[-1]=parts.pop()
                    res.pop()
                else:
                    parts = [next_vector(parts[0],vec)]
                    current = [parts[0]]
                    res = [tuple((vec[i]-parts[-1][i] for i in range(l)))] 
        except ValueError:
            if len(current)>1:
                current.pop()
                current[-1]=parts.pop()
                res.pop()
            else:
                yield VectorPartition([vec])
                return

def vecparn(vec, n, minvec=None):
    """
    Iterate over all ``n`` part vector partitions of ``vec``.

    INPUT:

    - ``vec`` -- tuple of non-negative integers.

    - ``minvec`` -- a tuple of nonnegative integers of the same length
      as ``vec`` and componentwise less than or equal to ``vec``.
    """
    if n==1:
        yield VectorPartition([vec])
        return
    l = len(vec)
    if minvec is None:
        minvec = (0,)*l
    parts = [minvec]
    res = [tuple((vec[i]-parts[-1][i] for i in range(l)))]
    current = [minvec]
    while True:
        try:
            v = next_vector(current[-1],res[-1])
            res.append(tuple(res[-1][i]-v[i] for i in range(l)))
            parts.append(v)
            current.append(v)
            if not any(res[-1]):
                if len(parts)==n:
                    yield VectorPartition(parts)
                current.pop()
                res.pop()
                parts.pop()
                if len(current)>1:
                    current.pop()
                    current[-1]=parts.pop()
                    res.pop()
                else:
                    parts = [next_vector(parts[0],vec)]
                    current = [parts[0]]
                    res = [tuple((vec[i]-parts[-1][i] for i in range(l)))]
            elif len(parts)==n:
                current.pop()
                res.pop()
                parts.pop()
                current[-1]=next_vector(current[-1],res[-1])
        except ValueError:
            if len(current)>1:
                current.pop()
                res.pop()
                parts.pop()
                current[-1]=next_vector(current[-1],res[-1])
            elif n==1:
                yield VectorPartition([vec])
            else:
                return


def q1(vec,n):
    return len(list(vecparn(vec,n)))

def w(k,l,n):
    """
    Return the multiplicity of the sign character of Sn in W(k,l)(n).
    """
    if l > 0:
        return q1((k,l),n)-q1((k+1,l-1),n)
    else:
        return q1((k,l),n)

def table_two_row(N):
    from numpy import zeros
    A = zeros([N,N,N],dtype=int)
    for n in range(N):
        for k in range(N):
            for l in range(k+1):
                A[k,l,n]=w(k,l,n)
    return A

def table_hooks(N):
    from numpy import zeros
    A = zeros([N,N,N],dtype=int)
    for n in range(N):
        for k in range(N):
            for l in range(N):
                A[k,l,n]=sign_mult(Partition([k+1]+[1]*l),n)
                return A

def sign_mult(la,n,sgn=1):
    """
    Return the multiplicity of the sign rep of Sn in W_la(n).
    """
    l = len(la)
    return sum([(w.sign()^sgn)*q1(tuple(la[i]-(i+1)+w(i+1) for i in range(l)), n) for w in Permutations(l)])

def table_two_col(N):
    from numpy import zeros
    A = zeros([N,N],dtype=int)
    for n in range(N):
        for k in range(1,N):
            for l in range(k+1):
                A[k,l,n]=sign_mult(Partition([k,l]).conjugate(),n)
    return A

def PR(n):
    """
    Return the polynomial ring over rationals with variables 0..n and dictionary of variables.

    EXAMPLE::
    
        sage: R, d = PR(4)
        sage: x[0]*x[2]-x[4]^2
        x0*x1 - x4^2
    """
    if n==0:
        R = PolynomialRing(QQ,'x',2)
    else:
        R = PolynomialRing(QQ,'x',n+1)
    return R, dict(enumerate(R.gens()))

class CharacterPolynomial():
    """
    Class of Character polynomials.

    EXAMPLES::

        sage: CharacterPolynomial()
        {}
        sage: CharacterPolynomial({Partition([2,1]):1, Partition([2,2,1]):1})
        {[2, 2, 1]: 1, [2, 1]: 1}
    """
    def __init__(self, p=dict(), algo=None):
        """
        Construct ``self`` from dictionary, partition, or polynomial.

        INPUT:

        ``p`` - a dictionary whose keys are partitions and values are scalars or a
            partition for which the character polynomial is sought, or 
        
        NOTE: The character polynomial is the sum over all keys (which are partitions)
        of `\binom{X_1}{m_1}\binom{X_2}{m_1}...` where `m_i` is the number of parts
        equal to ``i`` in the partition, multiplied by the corresponding value.
        """
        if isinstance(p, dict):
            self.dict = p
            self.n = max([max(la+[0]) for la in self.dict.keys()]+[1])
        elif isinstance(p, Partition) or isinstance(p, list):
            u = CharacterPolynomial(char_poly_weyl(p))
            self.dict = u.dict
            self.n = u.n
        else:
            d = dict(enumerate(p.parent().gens()))
            self.n = p.parent().ngens()-1
            if algo == "old":
                self.dict = {}
                while p!=0:
                    m = p.monomials()[0]
                    degs = m.degrees()
                    den = prod([factorial(deg) for deg in degs],1)
                    c = p.coefficients()[0]*den
                    p = p - prod([binomial(d[i],degs[i]) for i in range(self.n+1)], 1)*c
                    self.dict[Partition(exp=degs[1:])]=c
            self.dict = binomial_basis_expansion(p)
    def __repr__(self):
        return str(self.dict)
    def __eq__(self,other):
        return self.dict==other.dict
    def pp(self,n=None):
        """
        Return LaTeX code for ``self``.
        """
        output = str()
        keys = sorted(self.dict.keys(),key=lambda x:sum(x),reverse=True)
        for k in keys:
            if self.dict[k]==1:
                output += '+ \\binom{X}{'+str(k.to_exp(n))[1:-1]+'}'
            elif self.dict[k]==-1:
                output += '- \\binom{X}{'+str(k.to_exp(n))[1:-1]+'}'
            else:
                output += '+ ' + latex(self.dict[k]) +' \\binom{X}{'+str(k.to_exp(n))[1:-1]+'}'
            if output[0] == '+':
                output = output[2:]
        return output
    def degree(self):
        """
        Return degree of ``self``.
        """
        return max([k.size() for k in self.dict.keys()])
    def coeff(self,la):
        if la in self.dict.keys():
            return self.dict[la]
        else:
            return 0
    def polynomial(self,n=None):
        """
        Return the polynomial associated to ``self`` in ``n`` variables.
        """
        if n is None:
            n = self.n
        R,x = PR(n)
        p = 0
        for la in self.dict.keys():
            e = la.to_exp()
            p += prod([binomial(x[i+1],b) for i,b in enumerate(e)],R(1))*self.dict[la]
        return p
    def value(self,beta):
        """
        Return the value of ``self`` at partition ``alpha``.
        """
        alpha=Partition(beta)
        return sum([partition_binomial(beta,alpha)*self.dict[alpha] for alpha in self.dict.keys()])
    def moment(self,n=None,signed=False):
        """
        Return the moment of ``self``.
        """
        if n is None:
            n = self.degree()
        if signed:
            return sum([self.dict[la]/la.centralizer_size()*la.sign() for la in self.dict.keys() if la.size() in (n,n-1)])
        else:
            return sum([self.dict[la]/la.centralizer_size() for la in self.dict.keys() if la.size()<=n])
    def sum(self,other):
        """
        Return the sum of ``self`` and ``other``.
        """
        n = max(self.n,other.n)
        return CharacterPolynomial(self.polynomial(n=n)+other.polynomial(n=n))
    def product(self,other):
        """
        Return the product of ``self`` and ``other``.
        """
        n = max(self.n,other.n)
        return CharacterPolynomial(self.polynomial(n=n)*other.polynomial(n=n))
        
def choice(la):
    return CharacterPolynomial({Partition(la):1})

def multichoice(la):
    la = Partition(la)
    R, x = PR(max(la))
    laexp = la.to_exp()
    return CharacterPolynomial(prod([binomial(x[i+1]+e-1,e) for i,e in enumerate(laexp)]))

def multi_to_bin(a):
    def coeff(k,l):
        if k == 0:
            return 1
        else:
            return binom(k-1,l-1)
    d = dict()
    l = len(a)
    for m in product(*[range(1,i+1) for i in a]):
        d[m]=prod([coeff(a[i],m[i]) for i in range(l)], 1)
    return d

def q(la):
    la = Partition(la)
    R, x = PR(la.size())
    L = []
    for k in range(la.size()+1):
        L += [mu.conjugate() for mu in la.conjugate().remove_horizontal_border_strip(k)]
    return sum([(-1)^(la.size()-mu.size())*
             sum([charval(mu,rho)*choice(rho).polynomial(n=la.size()) for rho in Partitions(sum(mu))]) for mu in L])

def umbralize(p):
    """
    Return the umbalization of ``p``.

    NOTE:

    The umbralization of a polynomial replaces each monomial
    `x_1^{a_1}... x_k^{a_k}` with 
    `\binom{x_1}{a_1}...\binom{x_k}{a_k}`.
    """
    def umbral_monomial(degs):
        return prod([prod([x[i]-j for j in range(d)]) for i,d in enumerate(degs)])
    x = dict(enumerate(p.parent().gens()))
    mc = zip(map(lambda x:x.degrees(), p.monomials()), p.coefficients())
    return sum([c*umbral_monomial(degs) for degs, c in mc])

def char_poly_specht(mu):
    """
    Return the character polynomial of the Specht module of ``la``.

    NOTE:

    See Proposition I.1 of Garsia and Goupil (EJC 16(2), 2009, R19).
    """
    mu = Partition(mu)
    k = max(mu.size(),2)
    R,x = PR(k)
    if mu.size()==0:
        return R(1)
    if mu.size()==1:
        return x[1]-1
    p = sum([charval(mu,alpha)*prod([((i+1)*x[i+1]-1)^a for i,a in enumerate(alpha.to_exp(k))])/alpha.centralizer_size() for alpha in Partitions(k)])
    return umbralize(p)

def mult_specht_in_weyl(la, mu, n=None, method="ext"):
    """
    Return the multiplicity of the `V_mu` in `W_la`.
    """
    return CharacterPolynomial(char_poly_specht(mu)*char_poly_weyl(la, method=method)).moment()

def charval(la,mu):
    if sum(la)==0 or sum(mu)==0:
        return 1
    else:
        return symmetrica.charvalue(la,mu)

def char_poly_sym(k,R=None):
    """
    Return the character polynomial of Sym^k.
    """
    if R is None:
        R = PolynomialRing(QQ,'x',k+1)
    if isinstance(k, Partition):
        return prod([char_poly_sym(a) for a in la])
    output = R(0)
    for la in Partitions(k):
        la_exp = la.to_exp(k)
        output+=prod([binomial(R.gen(i+1)+la_exp[i]-1,la_exp[i]) for i in range(k)],1)
    return output

def char_poly_ext(k, R=None):
    """
    Return the character polynomial of Alt^k.
    """
    if R is None:
        R = PolynomialRing(QQ,'x',k+1)
    output = R(0)
    for la in Partitions(k):
        la_exp = la.to_exp(k)
        output += (-1)^sum(la_exp[1::2])*prod([binomial(R.gen(i+1),la_exp[i]) for i in range(k)],1)
    return output
    
def char_poly_weyl(la, h=char_poly_sym, e=char_poly_ext, method="ext"):
    """
    Return the character polynomial of the Weyl module associated to ``la``.
    """
    if len(la)==0:
        return 1
    la = Partition(la)
    N = max(la)+len(la)-1
    R = PolynomialRing(QQ,'x',N+1)
    if method=="sym":# use Jacobi
        n = len(la)
        def entry(i,j):
            if la[i]+j-i < 0:
                return 0
            else:
                return h(la[i]+j-i,R=R)
    elif method=="ext":# use Trudi
        n = max(la)
        mu=la.conjugate()
        def entry(i,j):
            if mu[i]-i+j < 0:
                return 0
            else:
                return e(mu[i]-i+j)
    else:# use Giambelli
        n = la.frobenius_rank()
        a, b = la.frobenius_coordinates()
        def entry(i,j):
            k, l = a[i], b[j]
            return sum([char_poly_sym(k+s+1,R=R)*char_poly_ext(l-s,R=R)*(-1)^s for s in range(l+1)])
    A = Matrix(R,n,n,entry)
    return A.determinant()

def partition_binomial(beta,alpha):
    """
    Return the product of ``binom(bi,ai)`` where bi and ai are the number of i cycles.
    """
    B = beta.to_exp()
    A = alpha.to_exp()
    m = min(len(A), len(B))
    return prod([binomial(B[i],A[i]) for i in range(m)])

def binomial_basis_expansion(p):
    output={}
    R = p.parent()
    x = dict(enumerate(R.gens()))
    q = sum([p.dict()[k]*prod([sum([stirling_number2(k[i],r)*x[i]^r*factorial(r) for r in range(k[i]+1)]) for i in range(len(k))]) for k in p.dict().keys()])
    return dict([(Partition(exp=k[1:]),q.dict()[k]) for k in q.dict().keys()])


***Code to compare the values from both methods for a range of values of $l$ and $k$.***

In [10]:
n=7
range_k=10
for l in range(n):
    for k in range(range_k):
        x=Partition([k+1]+[1]*l)
        print(x,' sign:',sign_mult(x,n)==sgn_mult_hook(k,l,n))
        print(x,' Triv:',CharacterPolynomial(char_poly_weyl([k+1]+[1]*l)).moment(n)==triv_mult_hook(k,l,n))

[1]  sign: True
[1]  Triv: True
[2]  sign: True
[2]  Triv: True
[3]  sign: True
[3]  Triv: True
[4]  sign: True
[4]  Triv: True
[5]  sign: True
[5]  Triv: True
[6]  sign: True
[6]  Triv: True
[7]  sign: True
[7]  Triv: True
[8]  sign: True
[8]  Triv: True
[9]  sign: True
[9]  Triv: True
[10]  sign: True
[10]  Triv: True
[1, 1]  sign: True
[1, 1]  Triv: True
[2, 1]  sign: True
[2, 1]  Triv: True
[3, 1]  sign: True
[3, 1]  Triv: True
[4, 1]  sign: True
[4, 1]  Triv: True
[5, 1]  sign: True
[5, 1]  Triv: True
[6, 1]  sign: True
[6, 1]  Triv: True
[7, 1]  sign: True
[7, 1]  Triv: True
[8, 1]  sign: True
[8, 1]  Triv: True
[9, 1]  sign: True
[9, 1]  Triv: True
[10, 1]  sign: True


KeyboardInterrupt: 